In [ ]:
import pandas as pd
import numpy as np
import os
import sqlite3

In [2]:
db_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'datalab.sqlite')
db_path

'/home/ronan/code/RonanB400/Project/decp_ml/data/datalab.sqlite'

In [3]:
conn = sqlite3.connect(db_path)
db = conn.cursor()
query = f"""
SELECT *
FROM "data.gouv.fr.2022.clean"
"""

data_raw = pd.read_sql_query(query, conn)
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299670 entries, 0 to 299669
Data columns (total 31 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   uid                        299670 non-null  object 
 1   id                         299670 non-null  object 
 2   nature                     299670 non-null  object 
 3   acheteur_id                299670 non-null  object 
 4   acheteur_nom               299477 non-null  object 
 5   acheteur_siren             299500 non-null  object 
 6   titulaire_id               299670 non-null  object 
 7   titulaire_typeIdentifiant  299670 non-null  object 
 8   titulaire_nom              292097 non-null  object 
 9   titulaire_siren            298537 non-null  object 
 10  objet                      299670 non-null  object 
 11  montant                    299670 non-null  float64
 12  codeCPV                    299670 non-null  object 
 13  procedure                  29

In [26]:
cpv_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'cpv_2008_ver_2013_FR.csv')
df_cpv = pd.read_csv(cpv_path)

'/home/ronan/code/RonanB400/Project/decp_ml/data/cpv_2008_ver_2013_FR.csv'

In [29]:
data_cpv = pd.merge(data_raw, df_cpv, left_on='codeCPV', right_on='CODE', how='left')
data_cpv.rename(columns={'FR': 'codeCPV_FR'}, inplace=True)
data_cpv[['codeCPV', 'codeCPV_FR', 'CODE']].head(10)

,codeCPV,codeCPV_FR,CODE
0,45210000-2,Travaux de construction de bâtiments,45210000-2
1,45261000-4,Travaux de charpente et de couverture et trava...,45261000-4
2,72514200-3,Services de gestion d'installations pour le dé...,72514200-3
3,71300000,NaN,NaN
4,44316500-3,Serrurerie,44316500-3
5,45261000-4,Travaux de charpente et de couverture et trava...,45261000-4
6,45212212,NaN,NaN
7,09310000-5,Électricité,09310000-5
8,45233142-6,Travaux de réparation de routes,45233142-6
9,33696500-0,Réactifs de laboratoire,33696500-0


In [30]:
data_cpv['codeCPV_FR'].isna().sum()

77370

In [59]:
missing_cpv = pd.DataFrame(data_cpv[data_cpv['codeCPV_FR'].isna() == True]['codeCPV'].unique())
missing_cpv.columns = ['codeCPV']
missing_cpv.head(10)

,codeCPV
0,71300000
1,45212212
2,15897300
3,79416200
4,45211350
5,45261000
6,39830000
7,71200000
8,16311100
9,90480000


In [53]:
df_cpv[df_cpv['CODE'].str.startswith('71300000')]['CODE'].values[0]

'71300000-1'

In [54]:
missing_cpv.loc[:, 'count_similar'] = missing_cpv['codeCPV'].astype(str).apply(
    lambda cpv: df_cpv['CODE'].str.startswith(cpv).sum()
)

missing_cpv.loc[:, 'new_CPV'] = missing_cpv['codeCPV'].astype(str).apply(
    lambda cpv: df_cpv[df_cpv['CODE'].str.startswith(cpv)]['CODE'].values[0] if 
    df_cpv[df_cpv['CODE'].str.startswith(cpv)].shape[0] > 0 else None
)

missing_cpv.head()

,codeCPV,count_similar,new_CPV
0,71300000,1,71300000-1
1,45212212,1,45212212-5
2,15897300,1,15897300-5
3,79416200,1,79416200-5
4,45211350,1,45211350-7


In [50]:
missing_cpv['count_similar'].value_counts()

count_similar
1    2921
0     622
Name: count, dtype: int64

In [57]:
no_cpv = missing_cpv[missing_cpv['count_similar'] == 0]['codeCPV']
no_cpv

12      20310000-2
15      74224000-5
16      07124000-2
20      74200000-1
28      29715000-4
           ...    
3516    45300000-9
3522    74231410-4
3525    45232411-0
3535    33263200-3
3542    66150000-8
Name: codeCPV, Length: 622, dtype: object

In [60]:
data_cpv[data_cpv['codeCPV'].isin(no_cpv)][['codeCPV', 'codeCPV_FR', 'CODE']]

,codeCPV,codeCPV_FR,CODE
46,20310000-2,NaN,NaN
52,74224000-5,NaN,NaN
57,07124000-2,NaN,NaN
70,74200000-1,NaN,NaN
101,29715000-4,NaN,NaN
...,...,...,...
299292,28812200-9,NaN,NaN
299414,74141520-0,NaN,NaN
299491,66150000-8,NaN,NaN
299515,74131500-1,NaN,NaN


In [ ]:
data_cpv = pd.merge(data_cpv, missing_cpv[['codeCPV', 'new_CPV']], left_on='codeCPV', right_on='codeCPV', how='left')
